In [1]:
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers==3
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 46.9MB 109kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 2.8MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595676 sha256=890ceb97dc8fa73b28fe9947e655830090cb5d4d89db86606b2af2f2c5319f93
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 2.8MB/s 
     |████████████████████████████████| 757kB 2.9MB/s 
     |████████████████████████████████| 901kB 17.0MB/s 
     |████████████████████████████████| 3.0MB 18.8MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-36t_52n4
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-36t_52n4
  Created wheel for kobert

In [ ]:
import pandas as pd
import os
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optims
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import subprocess

device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()



dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/애쓰는 감자/data/mid_data/train_mid_1.txt', field_indices=[1,2], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset('/content/tst.txt', field_indices=[1,2], num_discard_samples=1)


tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)



class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


max_len = 256 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5


data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)


# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
# test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)



class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 77, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


model = BERTClassifier(bertmodel, dr_rate=0.2).to(device)
# model = torch.load('weights/last_kobert_pytorch_model_big2s.pt')

# if torch.cuda.device_count() > 1:
    # model = nn.DataParallel(model)
# model = nn.DataParallel(model, output_device=[0,1])
# model.to(device)


# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]



# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# lr_scheduler = optims.lr_scheduler.CosineAnnealingLR(optimizer,T_max=0.1,eta_min=0.0001)


# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc



# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    best_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        # lr_scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
        if train_acc > best_acc:
            best_acc = train_acc
        torch.save(model, '/content/drive/MyDrive/애쓰는 감자/data/minz_data/weight/best_kobert_mid_1.pt')
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

torch.save(model, '/content/drive/MyDrive/애쓰는 감자/data/minz_data/weight/last_kobert_mid_1.pt')


# # model.eval() # 평가 모드로 변경
    
# # for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
# #     token_ids = token_ids.long().to(device)
# #     segment_ids = segment_ids.long().to(device)
# #     valid_length= valid_length
# #     label = label.long().to(device)
# #     out = model(token_ids, valid_length, segment_ids)
# #     test_acc += calc_accuracy(out, label)
# # print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

using cached model
using cached model
using cached model


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


epoch 1 batch id 1 loss 4.381725311279297 train acc 0.03125
epoch 1 batch id 201 loss 4.042562484741211 train acc 0.09235074626865672
epoch 1 batch id 401 loss 3.3187170028686523 train acc 0.12951995012468828
epoch 1 batch id 601 loss 3.164393663406372 train acc 0.18625207986688852
epoch 1 batch id 801 loss 2.387950897216797 train acc 0.24804931335830213
epoch 1 batch id 1001 loss 2.341085910797119 train acc 0.29879495504495507
epoch 1 batch id 1201 loss 2.277179718017578 train acc 0.34120004163197337
epoch 1 batch id 1401 loss 1.6469964981079102 train acc 0.376628301213419
epoch 1 batch id 1601 loss 1.7877960205078125 train acc 0.4069331667707683
epoch 1 batch id 1801 loss 1.5822762250900269 train acc 0.43104525263742366
epoch 1 batch id 2001 loss 1.3391884565353394 train acc 0.4528673163418291
epoch 1 batch id 2201 loss 1.2846425771713257 train acc 0.4721007496592458
epoch 1 batch id 2401 loss 1.5218197107315063 train acc 0.48832517700957934
epoch 1 batch id 2601 loss 1.7728384733200

<font color = pink> test 예측하기 </font>

In [ ]:
# test 예측하기
device = torch.device('cuda:0')
model = torch.load('/content/drive/MyDrive/애쓰는 감자/data/minz_data/best_kobert_model_small.pt')
model.to(device)
# model = nn.DataParallel(model, output_device=[0,1])
bertmodel, vocab = get_pytorch_kobert_model()
model.eval() # 평가 모드로 변경
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
max_len = 256 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5
dataset_test = nlp.data.TSVDataset('/content/drive/MyDrive/애쓰는 감자/data/new_data/bert_test.txt', field_indices=[1,2], num_discard_samples=1)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
test_acc = 0.0
df = pd.DataFrame(columns=['pred','label'])
pred = np.array([])
answer = np.array([])
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    _,max_idx = torch.max(out,1)
    pred = np.append(pred,max_idx.cpu().detach().tolist())
    # answer = np.append(answer,label.cpu().detach().tolist())
    # test_acc += calc_accuracy(out, label)
    # print(len(pred))
df['pred'] = pred
# df['label'] = answer

df.to_csv('/content/drive/MyDrive/애쓰는 감자/data/minz_data/test_small.csv',index=False)
# print("test acc {}".format(test_acc / (batch_id+1)))

using cached model
using cached model
using cached model


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


<font color = pink> val 예측하기 </font>

In [ ]:
# validation 예측하기

device = torch.device('cuda:0')
model = torch.load('/content/drive/MyDrive/애쓰는 감자/data/minz_data/weight/best_kobert_model_small.pt')
model.to(device)
# model = nn.DataParallel(model, output_device=[0,1])
bertmodel, vocab = get_pytorch_kobert_model()
model.eval() # 평가 모드로 변경
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
max_len = 256 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5
dataset_test = nlp.data.TSVDataset('/content/drive/MyDrive/애쓰는 감자/data/minz_data/val_small_pred.txt', field_indices=[1,2], num_discard_samples=1)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
test_acc = 0.0
df = pd.DataFrame(columns=['pred','label'])
pred = np.array([])
answer = np.array([])
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    _,max_idx = torch.max(out,1)
    pred = np.append(pred,max_idx.cpu().detach().tolist())
    # answer = np.append(answer,label.cpu().detach().tolist())
    # test_acc += calc_accuracy(out, label)
    # print(len(pred))
df['pred'] = pred
# df['label'] = answer

df.to_csv('/content/drive/MyDrive/애쓰는 감자/data/minz_data/val_small_pred.csv',index=False)
# print("test acc {}".format(test_acc / (batch_id+1)))

using cached model
using cached model
using cached model


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
